In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
from torch.utils.tensorboard import SummaryWriter
import random

/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:544: FutureWa

In [60]:
class Arguments():
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 64
        self.epochs = 30
        self.lr = 0.01
        self.no_cuda = False
        self.seed = 1
        self.log_interval = 60
        self.save_model = False

args = Arguments()

use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [61]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('~/data', train=True, download=True, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.batch_size
)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('~/data', train=False, download=True, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.test_batch_size
)


In [62]:
len(train_loader.dataset)

60000

In [63]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 34, 5, 1)
        self.conv2 = nn.Conv2d(34, 64, 5, 1)
        self.fc1 = nn.Linear(20*20*64, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = x.view(-1, 20*20*64)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

# model is not exactully the same as the paper since it did not mention the unit of fc


In [64]:
def train(args, device, model, train_dataloader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_dataloader):
        data,target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        
        # Chose the best step size(learning rate)
        optimizer.step()
        
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * args.batch_size, len(train_dataloader.dataset) ,
                100. * batch_idx * args.batch_size / len(train_dataloader.dataset), loss.item()))

In [65]:
def test(args, device, model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader)*(args.batch_size)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / (len(test_loader.dataset))))

In [66]:
#%%time
model = Net().to(device)
optimizer =  optim.SGD(model.parameters(), lr=args.lr)


for epoch in range(1, args.epochs + 1):
    train(args, device, model, train_loader,optimizer , epoch)
    test(args, device, model, test_loader)

if (args.save_model):
    torch.save(model.state_dict(), "mnist_cnn.pt")

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.308238
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.398305
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.394937
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.591041
Train Epoch: 1 [15360/60000 (26%)]	Loss: 0.263656
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.334817
Train Epoch: 1 [23040/60000 (38%)]	Loss: 0.264997
Train Epoch: 1 [26880/60000 (45%)]	Loss: 0.409800
Train Epoch: 1 [30720/60000 (51%)]	Loss: 0.250801
Train Epoch: 1 [34560/60000 (58%)]	Loss: 0.242357
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.160405
Train Epoch: 1 [42240/60000 (70%)]	Loss: 0.051608
Train Epoch: 1 [46080/60000 (77%)]	Loss: 0.254579
Train Epoch: 1 [49920/60000 (83%)]	Loss: 0.143675
Train Epoch: 1 [53760/60000 (90%)]	Loss: 0.111533
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.242175

Test set: Average loss: 0.1462, Accuracy: 9536/10000 (95%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.075186
Train Epoch: 2 [3840/60000 (6%)]	Loss: 0.051399
Train Epoch: 2 [7680/60000 (13%)]	Loss: 0.127741
Train

KeyboardInterrupt: 